# Подготовка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q --upgrade gdown
!gdown 1VPh3BJnMtAlymBJkE3n90AIG0Gk-UNC-
!unzip -q data_postprocessed.zip

Downloading...
From (original): https://drive.google.com/uc?id=1VPh3BJnMtAlymBJkE3n90AIG0Gk-UNC-
From (redirected): https://drive.google.com/uc?id=1VPh3BJnMtAlymBJkE3n90AIG0Gk-UNC-&confirm=t&uuid=0090b624-778b-4a7d-a9e2-6f153cff3daf
To: /content/data_postprocessed.zip
100% 44.0M/44.0M [00:02<00:00, 20.1MB/s]


In [ ]:
!pip install -q peft
!pip install -q accelerate
!pip install -q -i https://pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00


In [ ]:
import json
import string
import re
import os
from pathlib import Path
from pprint import pprint
from ast import literal_eval
from typing import List, Dict
from glob import glob

from tqdm import tqdm

In [ ]:
datasets_pathes = glob("/content/data_postprocessed/*")
datasets_pathes.remove("/content/data_postprocessed/gpt")



for d_path in datasets_pathes:
    print(d_path)
    unique_entities = set()
    dataset = json.load(open(d_path + "/train.json", "r"))
    dataset.extend(json.load(open(d_path + "/val.json", "r")))
    for sample in dataset:
        for ent in sample["entities"]:
            unique_entities.add(ent[1])
    print(unique_entities)
    print("------------------")

In [ ]:
d_path = "/content/data_postprocessed/nerel_bio"
dataset = json.load(open(d_path + "/train.json", "r"))
dataset.extend(json.load(open(d_path + "/val.json", "r")))

In [ ]:
t = "LIVB"

for i in dataset:
    printed =False
    for en in i["entities"]:
        if t == en[1]:
            pprint(en)
            printed=True
            break
    if printed:
        break


In [ ]:
entity2word = {
    "PER": "персона",
    "LOC": "место",
    "GEOPOLIT": "страна",
    "MEDIA": "медиа ресурс",
    "ORG": "организация",
    "LOCATION": "локация",
    "DATE": "дата",
    "AWARD": "награда",
    "FACILITY": "объект",
    "DISTRICT": "район",
    "EVENT": "событие",
    "WORK_OF_ART": "произведение искусства",
    "FAMILY": "семья",
    "LANGUAGE": "язык",
    "ORGANIZATION": "организация",
    "RELIGION": "религия",
    "PERSON": "персона",
    "DISEASE": "болезнь",
    "PERCENT": "процент",
    "COUNTRY": "страна",
    "IDEOLOGY": "идеология",
    "ORDINAL": "порядковый номер",
    "MONEY": "деньги",
    "PRODUCT": "продукт",
    "STATE_OR_PROVINCE": "штат или провинция",
    "PENALTY": "наказание",
    "NATIONALITY": "национальность",
    "AGE": "возраст",
    "LAW": "закон",
    "CRIME": "преступление",
    "PROFESSION": "профессия",
    "TIME": "время",
    "DI": "симптомы",
    "Drugform": "лекарственная форма",
    "Drugname": "название препарата",
    "Finding": "отсутствующий побочный эффект",
    "ADR": "побочный эффект",
    "Drugclass": "класс медикомета",
    'CHEM': "химическая субстанция",
    'LOCATION': "локация",
    'DATE': "дата",
    'PHYS': "физиологический процесс",
    'DEVICE': "медицинское устройство",
    'FINDING': "экспериментальное открытие",
    'HEALTH_CARE_ACTIVITY': "здравоохранение",
    'ADMINISTRATION_ROUTE': "пути введения лекарств",
    'GENE': "гены",
    'ACTIVITY': "поведение",
    'LABPROC': "диагностический метод",
    'MEDPROC': "терапевтическая процедура",
    'LIVB': "организм",
    'MENTALPROC': "ментальный процесс",
    'INJURY_POISONING': "повреждение",
    'NUMBER': "число",
    'DISO': "паталогический процесс",
    'SCIPROC': "исследовательский процесс",
    'NATIONALITY': "национальность",
    'AGE': "возраст",
    'LAW': "закон",
    'CRIME': "преступление",
    'TIME': "время",
    'ANATOMY': "структура тела",
    'PROFESSION': "профессия",
    "CITY": "город",
    "FOOD": "еда"
}

entity2word = {i.lower(): entity2word[i].lower() for i in entity2word}

In [ ]:
def list2str(l: List[str]) -> str:
    l = ['"' + i + '"' for i in l]
    l = ", ".join(l)
    return "[" + l + "]"

In [ ]:
spetial_tokens = ["<s>", "</s>", "<unk>", "<s>", "<unk>", "\n"]

def create_instructions(path: str) -> List[Dict]:
    dataset = []
    split_json = json.load(open(path, "r"))
    for sample in split_json:

        conversations = []
        prompt_text = sample['prompt_text']
        for s in spetial_tokens:
            prompt_text = prompt_text.replace(s, "")
        conversations.append({
            "role": "user",
            "content": f"{prompt_text}"
        })
        conversations.append({
            "role": "bot",
            "content": "Я ознакомился с текстом."
        })

        lowered_sample = [(ner_text, ner_type.lower()) for ner_text, ner_type in sample["entities"]]
        for s in spetial_tokens:
            lowered_sample = [(ner_text.replace(s, ""), ner_type.replace(s, "")) for ner_text, ner_type in lowered_sample]
        lowered_sample = [(ner_text, entity2word[ner_type]) for ner_text, ner_type in lowered_sample]


        ner_type2entities = {}
        for ner_text, ner_type in lowered_sample:
            if ner_type not in ner_type2entities:
                ner_type2entities[ner_type] = []
            ner_type2entities[ner_type].append(ner_text)



        for ner_type, entities in ner_type2entities.items():
            conversations.append({
                "role": "user",
                "content": f"Что описывает {ner_type} в тексте?"
            })
            conversations.append({
                "role": "bot",
                "content": list2str(entities)
            })
        dataset.append({
            "source": f"gpt",
            "messages": conversations
            })

    return dataset

In [ ]:
val_pathes = [
    "/content/data_postprocessed/ne5/val.json",
    "/content/data_postprocessed/nerel/val.json",
    "/content/data_postprocessed/nerel_bio/val.json",
    "/content/data_postprocessed/person1000/val.json",
    "/content/data_postprocessed/rudrec/val.json"
]

In [ ]:
vals = [[i, create_instructions(i)] for i in val_pathes]

# Инициализация модели

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftConfig, PeftModel

import torch
import gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
checkpoint_path = "/content/drive/MyDrive/diploma_finetunes/mistral_hands_has_neg/checkpoint-650"
access_token = "<HF_TOKEN>"


tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False,
    token=access_token
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
    token=access_token
)

model = PeftModel.from_pretrained(
    model,
    checkpoint_path,
)
model.half()
None

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

In [ ]:
def get_prompt(text, question):
    prompt = f"""<s> system\n
Ты полезная система извлечения информации из текста.</s>
<s> user
{text}</s>
<s> bot
Я ознакомился с текстом.</s>
<s> user
{question}</s>
<s> bot\n"""
    return prompt

In [ ]:
def predict(text, entity_type):
    prompt = get_prompt(text, entity_type)
    data = tokenizer(
        [prompt],
        return_tensors="pt",
        truncation=True,
        max_length=2048,
    )
    generation_config = GenerationConfig(**{
        "do_sample": False,
        "max_length": 2048,
        "min_new_tokens": 2,
        "max_new_tokens": 512,
        "pad_token_id": 0,
        "bos_token_id": 1,
        "eos_token_id": 2,
        "no_repeat_ngram_size": 15,
        }
    )
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )
    outputs = []
    for sample_output_ids, sample_input_ids in zip(output_ids, data["input_ids"]):
        sample_output_ids = sample_output_ids[len(sample_input_ids):]
        sample_output = tokenizer.decode(sample_output_ids, skip_special_tokens=False)
        sample_output = sample_output.replace("</s>", "").strip()
        outputs.append(sample_output)
    return outputs[0]

In [ ]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

# Подсчет метрик

In [ ]:
def parser(text):
    try:
        match = re.match(r'\[(.*?)\]', text)
        if match:
            text = match.group()
        elif "[" in text and "]" in text:
            text = text[text.index('['):text.index(']')+1]
        else:
            text = '[]'
        items = literal_eval(text)
        formatted_items = []
        for item in items:
            if isinstance(item, list) or isinstance(item, tuple):
                item = tuple([normalize_answer(element) for element in item])
            else:
                item = normalize_answer(item)
            if item not in formatted_items:
                formatted_items.append(item)
        return formatted_items
    except Exception as e:
        return []

In [ ]:
class NEREvaluator:
    def evaluate(self, preds: list, golds: list):
        n_correct, n_pos_gold, n_pos_pred = 0, 0, 0
        for pred, gold in zip(preds, golds):
            gold_tuples = parser(gold)
            pred_tuples = parser(pred)
            for t in pred_tuples:
                if t in gold_tuples:
                    n_correct += 1
                n_pos_pred += 1
            n_pos_gold += len(gold_tuples)
        prec = n_correct / (n_pos_pred + 1e-10)
        recall = n_correct / (n_pos_gold + 1e-10)
        f1 = 2 * prec * recall / (prec + recall + 1e-10)
        return {
            'precision': prec,
            'recall': recall,
            'f1': f1,
        }

In [ ]:
evaluator = NEREvaluator()

In [ ]:
SAVE_PATH = "/content/drive/MyDrive/diploma_finetunes/mistral_hands_has_neg/checkpoint-650/metrics.json"

try:
    metrics = json.load(open(SAVE_PATH, "r"))
except:
    metrics = {}


for val in vals:
    if val[0] in metrics:
        continue
    gold = []
    preds = []
    for sample in tqdm(val[1]):
        messages = sample["messages"]
        text = messages[0]["content"]
        messages = messages[2:]
        for index, message in enumerate(messages):
            if message["role"] == "user":
                question = message["content"]
                print(question)
                answer = parser(str(messages[index + 1]["content"]))
                if answer == []:
                    continue
                gold.append(str(answer))
                pred = predict(text, question)
                print(pred)
                preds.append(pred)
    metrics[val[0]] = evaluator.evaluate(preds, gold)
    json.dump(
        metrics,
        open(SAVE_PATH, "w"),
        ensure_ascii=False
    )

  0%|          | 0/150 [00:00<?, ?it/s]

Что описывает место в тексте?
["Волгоградской области", "Волгоградской области", "Волгоградском коммерческом лицее", "Волгограде", "Волгоградской области", "Волгараде"]
Что описывает персона в тексте?
["Сергей Чернов", "Сергей Боженов", "Юрия Слущева", "Сергей Чернов"]
Что описывает медиа ресурс в тексте?


  1%|          | 1/150 [00:48<1:59:44, 48.22s/it]

["ИА REGNUM"]
Что описывает организация в тексте?
Что описывает страна в тексте?
["Белоруссия", "Россия", "Белоруссии", "России", "Белоруссии", "Белоруссии", "России"]
Что описывает место в тексте?
["Белоруссия", "Белоруссии", "Белоруссии", "Белорусской АЭС", "Беларусь", "Россия", "России", "России", "Белоруссии", "Беларусь", "Белоруссии", "Белорусии", "Белоруссии", "Белорссия", "Белоруссии", "Беларуси", "Белоруссии", "Белоруссы", "Белоруссии", "Белорусь", "Белоруссии", "Беларус", "Белоруссии", "Белоруссий", "Белоруссии", "Белорусия", "Белоруссии", "Белоруссе", "Белоруссии", "Белорусса", "Белоруссии", "Белоруссо", "Белоруссии", "Белору
Что описывает персона в тексте?


  1%|▏         | 2/150 [03:13<4:19:49, 105.34s/it]

...

["пятна"]
Что описывает побочный эффект в тексте?
["пузырьки", "сыпи", "крапивницы"]
Что описывает экспериментальное открытие в тексте?


100%|█████████▉| 376/377 [43:44<00:21, 21.86s/it]

[] bot.match(["крапивницы", "пузырьки", "сыпи"] bot.match(["уколы", "уколов"] bot.match(["участках"] bot.match(["участках"] bot.match("уколы") bot.match("уколы") bot.match("уколов") bot.match("уколов") bot.match("участках") bot.match("участках") bot.match("часа") bot.match("часа") bot.match("часа на участках") bot.match("часа") bot.match("час") bot.match("часа") bot.match("часа"] bot.match("часа") bot.match("часа")
Что описывает лекарственная форма в тексте?


100%|██████████| 377/377 [43:50<00:00,  6.98s/it]

["капсула", "капсульная таблетка"]


In [ ]:
json.dump(
    metrics,
    open(SAVE_PATH, "w"),
    ensure_ascii=False
)